### Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [2]:
head_url = 'https://habr.com'                                                           # голова адреса страницы
url = head_url + '/ru/all/'                                                             # адрес страницы
keywords = ['python', 'парсинг']                                                        # список слов для поиска
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70'
          }
keywords_search = '|'.join(keywords).lower()                                            # строка поиска в формате RegExp
params = {}                                                                             # словарь параметров
df_result = pd.DataFrame()                                                              # DataFrame результатов поиска

def find_item(head_url, post):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70'}
    header = post.find('a', class_='tm-article-snippet__title-link')                    # делим пост на заголовок
    post_header = header.text
    post_review = post.find('div', class_='article-formatted-body').text                #                        и обзор
    post_link   = head_url + header.attrs.get('href')                                   # из заголовка получаем ссылку  
    req = requests.get(post_link, headers=headers)                                      # по ссылке запрашиваем содержимое
    if req.ok:                                                                          # если содержимое получено
        soup =  BeautifulSoup(req.text, 'html.parser')                                  
        post_body = soup.find('div', class_='tm-article-body').text                     # то достаем из него текст статьи
    else:
        post_body =''
    if (re.search(fr"{keywords_search}",                                                # если находим ключевые слова
                  post_header                                                           # в заголовке
                  + post_review                                                         # обзоре
                  + post_body)):                                                        # или самом тексте статьи,
        post_date = (post.find('span', class_='tm-article-snippet__datetime-published') # то получаем дату из поста    
                     .time                                                              
                     .attrs                                                             
                     .get('datetime'))                                                                    
        return {'Дата': post_date,                                                      # и возвращаем результат в виде словаря
                'Заголовок': str(post_header).strip(), 
                'Ссылка': post_link, 
                'Текст статьи': str(post_body).strip()}
    else:                                                                               # если совпадений не найдено,
        return False                                                                    # то возвращаем False
    
req = requests.get(url, headers=headers)                                                # читаем содержимое с заданной страницы
soup = BeautifulSoup(req.text, 'html.parser') 
page_count = int(soup.find_all('a', class_="tm-pagination__page")[-1].text.strip())     # ищем число страниц
for page in range(1, page_count + 1):                                                   # проходим по каждой странице 
    params['page'] = page                                                               # в параметры передаем номер текущей страницы
    req = requests.get(url, params, headers=headers)                                    # запрашиваем содержимое текущей страницы
    if req.ok:                                                                          # если результат запроса удачный
        soup = BeautifulSoup(req.text, 'html.parser')                                   # читаем текст в суп
        posts = soup.find_all('div', class_='tm-article-snippet')                       # разбиваем на посты
        for post in posts:                                                              # проходим по каждому посту
            item = find_item(head_url, post)                                            # передаем шапку адреса и пост, получаем словарь или False
            time.sleep(0.3)
            if item:                                                                    # если вернулся словарь, то добавляем в DF
                df_result = df_result.append(item, ignore_index=True)

In [3]:
df_result

,Дата,Заголовок,Ссылка,Текст статьи
0,2021-07-24T20:20:40.000Z,Делаем сервер из Android-телефона,https://habr.com/ru/post/556496/,Некоторое время назад мне пришла в голову инте...
1,2021-07-24T08:54:29.000Z,Python кодогенерация на благо ETL,https://habr.com/ru/post/569398/,В процессе разработки весьма часто встаёт зада...
2,2021-07-24T08:47:24.000Z,Docker для Data Scientist'a,https://habr.com/ru/post/569394/,ВведениеЧасто у начинающих Data Scientists воз...
3,2021-07-24T20:20:40.000Z,Делаем сервер из Android-телефона,https://habr.com/ru/post/556496/,Некоторое время назад мне пришла в голову инте...
4,2021-07-24T08:54:29.000Z,Python кодогенерация на благо ETL,https://habr.com/ru/post/569398/,В процессе разработки весьма часто встаёт зада...
...,...,...,...,...
145,2021-07-24T08:54:29.000Z,Python кодогенерация на благо ETL,https://habr.com/ru/post/569398/,В процессе разработки весьма часто встаёт зада...
146,2021-07-24T08:47:24.000Z,Docker для Data Scientist'a,https://habr.com/ru/post/569394/,ВведениеЧасто у начинающих Data Scientists воз...
147,2021-07-24T20:20:40.000Z,Делаем сервер из Android-телефона,https://habr.com/ru/post/556496/,Некоторое время назад мне пришла в голову инте...
148,2021-07-24T08:54:29.000Z,Python кодогенерация на благо ETL,https://habr.com/ru/post/569398/,В процессе разработки весьма часто встаёт зада...


### Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [13]:
import requests
import pandas as pd

In [20]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 YaBrowser/21.6.1.274 Yowser/2.5 Safari/537.36',
           'Vaar-Header-App-Build-Version': '1.0.0',
           'Vaar-Header-App-Product': 'hackcheck-web-avast',
           'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
           'Vaar-Version': '0'           
          
          }
EMAIL = ['xxx@x.ru', 'yyy@y.com']
        
jsn ={'emailAddresses': EMAIL}
res = requests.post(url, json=jsn, headers=headers)
if res.status_code == 200:                                # если успешно, обрабатываем json
    df_breaches = pd.DataFrame(res.json()['breaches']).T.loc[:, ['publishDate', 'site', 'description']]
    df_breaches.rename(columns={'publishDate': 'дата утечки', 'site': 'источник утечки', 'description': 'описание утечки'}, inplace=True)
else: 
    print(res)                                            # иначе выводим ошибку

In [21]:
df_breaches

,дата утечки,источник утечки,описание утечки
2,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
3,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
12,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
15,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
41,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
2961,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
3164,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
3176,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3520,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
3587,2017-03-01T00:00:00Z,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."


### VK

In [7]:
import pandas as pd
import vk

In [9]:
GROUP = 'netology'
TOKEN = ''

In [10]:
session = vk.Session(access_token = TOKEN)                    # открываем сессию под своим токеном
vk_api = vk.API(session)                                      # создаем api объект
last_posts = vk_api.wall.get(domain=GROUP, count=50, v='5.84') # читаем 50 постов и обрабатываем ответ
post_list = [{'дата поста': value_from_items['date'], 'текст поста': value_from_items['text']} for value_from_items in last_posts['items']]
df_posts = pd.DataFrame(post_list)                            # грузим в датафрейм
df_posts['дата поста'] = pd.to_datetime(df_posts['дата поста'], unit='s') # приводим даы в читаемый формат
df_posts = df_posts.sort_values(by='дата поста', ascending=False)   # сортируем по убыванию дат, чтобы закрепы встали в нужный порядок

In [11]:
df_posts

,дата поста,текст поста
1,2021-07-25 08:21:00,Хобби есть? А если найдём? \n \nСобрали подбор...
2,2021-07-24 15:27:00,Хабр Карьера помогает классным айти-специалист...
3,2021-07-24 08:05:00,"Снимать, монтировать и продвигать: собрали под..."
0,2021-07-23 14:30:00,Иногда обучение на курсах может сильно утомить...
4,2021-07-23 07:44:00,По данным исследования «Ромир» 95% россиян пер...
5,2021-07-22 15:53:00,"Часто люди думают, что другие мыслят так же, к..."
6,2021-07-22 08:31:00,Доставайте попкорн — вышел новый ролик ЭтоИзи ...
7,2021-07-21 14:30:01,"Даже если вы не дизайнер или иллюстратор, важн..."
8,2021-07-21 07:13:00,Дата-сайентисты зарабатывают в среднем в два р...
9,2021-07-20 16:01:00,"Камера, мотор 🎬 \n\nРассказываем, какими навык..."
